In [ ]:
import pandas as pd
import nibabel as nib
import numpy as np
from radiomics import featureextractor
import SimpleITK as sitk

def read_mA_values_from_ods(ods_file):
    """Read mA values from an .ods file, assuming they are in the first column."""
    df = pd.read_excel(ods_file, engine='odf', header=None)
    return df[0].tolist()

def load_nifti_file(filepath):
    """Load a NIfTI file and return the data array."""
    nii = nib.load(filepath)
    return nii.get_fdata()

def apply_mask_and_compute_dose(dose, mask):
    """Apply organ mask to dose distribution and compute average dose."""
    masked_dose = dose * mask
    return np.mean(masked_dose[masked_dose > 0])

def extract_radiomic_features(image_path, mask_path, params_file):
    """Extract radiomic features using PyRadiomics and a YAML params file."""
    extractor = featureextractor.RadiomicsFeatureExtractor(params_file)
    image = sitk.ReadImage(image_path)
    mask = sitk.ReadImage(mask_path)
    features = extractor.execute(image, mask, label=1)
    return features

# Define file paths
params_file = '/path/to/radiomics_params.yaml'
ods_path = '/path/to/mean_mA_Breast_cohort.ods'

mA_values = read_mA_values_from_ods(ods_path)
all_features = []

# Main extraction loop
for i in range(1, 287):
    ct_path = f'/path/to/nifti_CT/Breast/{i}_0000.nii.gz'
    dose_path = f'/path/to/nifti_Dose/Breast/{i}.nii.gz'
    mask_path = f'/path/to/Breast_masks/{i}.nii.gz'

    dose_data = load_nifti_file(dose_path)
    mask_data = load_nifti_file(mask_path)

    average_dose = apply_mask_and_compute_dose(dose_data, mask_data)
    features = extract_radiomic_features(ct_path, mask_path, params_file)

    # Filter out diagnostics and append additional data
    features_dict = {k: v for k, v in features.items() if not k.startswith('diagnostics')}
    features_dict['mA'] = mA_values[i - 1]
    features_dict['average_dose (mGy)'] = average_dose * ((21.4 * mA_values[i - 1] * 0.4) / 100000)

    all_features.append(features_dict)

# Save results to csv and numpy 
df_features = pd.DataFrame(all_features)
df_features.to_csv('/path/to/Breast_data.csv', index=False)
np.save('/path/to/Breast_data.npy', df_features.to_numpy())
